In [ ]:
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler,AutoencoderKL
import torch
import os
import time
import pprint
import argparse
import numpy as np
import pickle
import utils
import csv

from model.hidden import Hidden
from noise_layers.noiser import Noiser
from average_meter import AverageMeter
from noise_argparser import NoiseArgParser

import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import json

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

save_folder='/tmp3/jimmyliu/HiDDeN_experiment/gen_images/test'

cap_path='/tmp3/jimmyliu/HiDDeN_experiment/cap_list.json'
caps=json.load(open(cap_path))
seeds_path='/tmp3/jimmyliu/HiDDeN_experiment/5000_seeds.json'
seeds=json.load(open(seeds_path))

In [ ]:
model_id = "runwayml/stable-diffusion-v1-5"
#model_id="stabilityai/stable-diffusion-2-1"
#stable_diffusion_txt2img = StableDiffusionPipeline.from_pretrained(model_id)


vae_path='/tmp3/jimmyliu/HiDDeN_experiment/finetune_SD/runs/3_no_w/checkpoints/vae_3'

# Use the DPMSolverMultistepScheduler (DPM-Solver++) scheduler here instead
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float32)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
tmpe_vae=AutoencoderKL.from_pretrained(vae_path)
pipe.vae.decoder=tmpe_vae.decoder
pipe.vae.post_quant_conv=tmpe_vae.post_quant_conv

def dummy(images, **kwargs):
    return images, False
pipe.safety_checker = dummy
pipe=pipe.to(device)
generator = torch.Generator(device=device)

In [ ]:
height=256
width=256
batch_size=2




In [ ]:
print(len(seeds)/batch_size)

In [ ]:
for i in range(int(len(seeds)/batch_size)):
    output=pipe(prompt=caps[i:i+batch_size],height=height,width=width,guidance_scale=3.0,generator = [generator.manual_seed(gen_seed) for gen_seed in seeds[i:i+batch_size]])
    for j in range(batch_size):
        output.images[j].save(os.path.join(save_folder,f'{i*batch_size+j}.png'))


In [ ]:
prompt = "Emma Watson as a powerful mysterious sorceress, casting lightning magic, detailed clothing, digital painting, hyperrealistic, fantasy, Surrealist, full body, by Stanley Artgerm Lau and Alphonse Mucha, artstation, highly detailed, sharp focus"
#temp = pipe(prompt=[prompt]*num_images,height=height,width=width,latents = latents)
temp = pipe(prompt=[prompt]*num_images,height=height,width=width,guidance_scale=3.0,generator = [generator.manual_seed(seeds[i]) for i in range (4)])

In [ ]:
images=temp.images

In [ ]:
img=images[0]
img

In [ ]:
from torchvision import  transforms
from noise_layers.jpeg_compression import rgb2yuv_tensor

In [ ]:
T=transforms.Compose([
            transforms.RandomCrop((512, 512), pad_if_needed=True),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
        ])

In [ ]:
img=T(images[0])

In [ ]:
print(img.shape)
img=img.unsqueeze(0)
print(img.shape)

In [ ]:
yuv_img=rgb2yuv_tensor(img)

In [ ]:
print(yuv_img.shape)

In [ ]:
fix_message=torch.Tensor([1., 1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0.,                                                                                                                                                                                                   
        1., 1., 0., 1., 0., 1., 1., 0., 1., 0., 0., 1., 1., 1., 0., 1., 1., 1.,                                                                                                                                                                                                                 
        0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.])

In [ ]:
ans=wm_decoder(yuv_img.to(device))
print(ans)

In [ ]:
decoded_rounded = ans.detach().cpu().numpy().round().clip(0, 1)

In [ ]:
print(decoded_rounded)

In [ ]:
bitwise_avg_err = np.sum(np.abs(decoded_rounded - fix_message.detach().cpu().numpy())) / fix_message.shape[1]